In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split

2023-09-18 01:40:54.605699: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-18 01:40:56.200816: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class NewsDataset(Dataset):
    def __init__(self, dataset):
        self.labels = dataset['label']
        self.input_ids = dataset['input_ids']
        self.attention_mask = dataset['attention_mask']

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx]
        input_ids = self.input_ids[idx]
        attention_mask = self.attention_mask[idx]
        return {"label": label, "input_ids":input_ids, "attention_mask":attention_mask}
    

def load_data(trainset_file_path, testset_file_path):
    train_data = pd.read_csv(trainset_file_path, encoding='utf-8')
    test_data = pd.read_csv(testset_file_path, encoding='utf-8')
    
    return train_data, test_data

def preprocessing_data(dataset, tokenizer, max_length):
    concat_entity = []
    for title, body in zip(dataset['Headline'], dataset['Content']):
        total = title + '[SEP]' + body
        concat_entity.append(total)
        
    tokenized_senteneces = tokenizer(
        concat_entity,
        return_tensors = "pt",
        padding = True,
        truncation = True,
        max_length = max_length,
        add_special_tokens = True,
        return_token_type_ids=False,
    )

    input_ids = tokenized_senteneces.input_ids
    attention_mask = tokenized_senteneces.attention_mask
    label = torch.tensor(dataset['Class'])
    
    return {
        "label": label,
        "input_ids": input_ids,
        "attention_mask": attention_mask,
    }

def get_dataloader(train_path, test_path, tokenizer, MAX_LEN, BATCH_SIZE):
    # 데이터 파일 로드
    train_data, test_data = load_data(train_path, test_path)
    
    # 토크나이징 & 데이터셋 로드
    tokenized_train = NewsDataset(preprocessing_data(train_data, tokenizer, MAX_LEN))
    tokenized_test = NewsDataset(preprocessing_data(test_data, tokenizer, MAX_LEN))
    
    # train, valid split
    generator = torch.Generator().manual_seed(RANDOM_SEED)
    train, valid = random_split(tokenized_train, [0.8, 0.2], generator=generator)
    
    # DataLoader
    train_dataloader = DataLoader(train, sampler=RandomSampler(train), batch_size=BATCH_SIZE, num_workers=8)
    valid_dataloader = DataLoader(valid, sampler=SequentialSampler(valid), batch_size=BATCH_SIZE, num_workers=8)
    test_dataloader = DataLoader(tokenized_test, sampler=SequentialSampler(tokenized_test), batch_size=BATCH_SIZE, num_workers=8)
    
    return train_dataloader, valid_dataloader, test_dataloader

In [42]:
import lightning.pytorch as pl
from collections import OrderedDict
from sklearn.metrics import accuracy_score, precision_score
import torchmetrics

In [73]:
class Model(pl.LightningModule):

    def __init__(self, MODEL_NAME):
        super(Model, self).__init__()
        model_config = AutoConfig.from_pretrained(MODEL_NAME)
        model_config.num_labels = 2
        model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, config=model_config)
        self.model = model

        self.accuracy = torchmetrics.classification.BinaryAccuracy()
        self.precision = torchmetrics.classification.BinaryPrecision()
        
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
                self.model.parameters(),
                lr=2e-5,
                )
        return optimizer

    def _shared_step(self, batch, batch_idx):
        labels = batch["label"]
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]

        output = self.model(
                input_ids,
                token_type_ids=None,
                attention_mask=attention_mask,
                labels=labels
                )
        return {'loss':output['loss'], 
                'labels':labels, 
                'logits': output['logits']
                }
        
    def training_step(self, batch, batch_idx):
        output = self._shared_step(batch, batch_idx)

        train_loss = output['loss']
        train_acc = self.accuracy(torch.argmax(output['logits'], dim=1), output['labels'])
        train_precision = self.precision(torch.argmax(output['logits'], dim=1), output['labels'])

        self.log('train_loss', train_loss)
        self.log('train_acc_step', train_acc)
        self.log('train_precision_step', train_precision)

        return output


    def validation_step(self, batch, batch_idx):
        output = self._shared_step(batch, batch_idx)

        valid_loss = output['loss']
        valid_acc = self.accuracy(torch.argmax(output['logits'], dim=1), output['labels'])
        valid_precision = self.precision(torch.argmax(output['logits'], dim=1), output['labels'])

        self.log('valid_loss', valid_loss)
        self.log('valid_acc_step', valid_acc)
        self.log('valid_precision_step', valid_precision)

        return output

    def test_step(self, batch, batch_idx):
        output = self._shared_step(batch, batch_idx)
        
        test_acc = self.accuracy(torch.argmax(output['logits'], dim=1), output['labels'])
        test_precision = self.precision(torch.argmax(output['logits'], dim=1), output['labels'])

        self.log('test_acc_step', test_acc)
        self.log('test_precision_step', test_precision)

        return output


In [74]:
model = Model('bert-base-multilingual-cased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [75]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
MAX_LEN = 450
BATCH_SIZE = 8
RANDOM_SEED = 42

In [76]:
train_dataloader, valid_dataloader, test_dataloader = get_dataloader('data_EC_00.csv', 'VL_data_EC.csv', tokenizer, MAX_LEN, BATCH_SIZE)

In [77]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [78]:
import wandb

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [79]:
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch.loggers import WandbLogger

In [83]:
wandb_logger = WandbLogger(project="mahimahi")
early_stop_callback = EarlyStopping(
    monitor='valid_loss',
    patience=2,
    verbose=True,
    mode='min'
)
model_checkpoint = ModelCheckpoint(
    dirpath='model/checkpoint',
)

callbacks = [early_stop_callback, model_checkpoint]
trainer = pl.Trainer(logger=wandb_logger,
                     check_val_every_n_epoch=1,
                     callbacks = callbacks,
                     max_epochs=10)
trainer.fit(model, train_dataloader, valid_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/aiffelmahimahi/anaconda3/envs/mahimahi/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /home/aiffelmahimahi/model/checkpoint exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                          | Params
------------------------------------------------------------
0 | model     | BertForSequenceClassification | 177 M 
1 | accuracy  | BinaryAccuracy                | 0     
2 | precision | BinaryPrecision               | 0     
------------------------------------------------------------
177 M     Trainable params
0         Non-trainable params
177 M     Total params
711.420   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Metric valid_loss improved. New best score: 0.678


...

Validation: 0it [00:00, ?it/s]

Metric valid_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.676


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Monitored metric valid_loss did not improve in the last 2 records. Best score: 0.676. Signaling Trainer to stop.


In [84]:
trainer.test(model, test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_acc_step       │    0.4972088634967804     │
│    test_precision_step    │    0.4972088634967804     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc_step': 0.4972088634967804,
  'test_precision_step': 0.4972088634967804}]